In [1]:
# analytics
import pandas as pd 
import numpy as np
import scipy.stats as stats
import statsmodels.formula.api as smf
#spatial 
import osmnx as ox
import geopandas as gpd
import contextily as cx
# plotting 
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
#settings
import warnings

In [2]:
# import data
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_n.csv'
df_n = pd.read_csv(path)
df_n.columns

Index(['id', 'state', 'zipcode', 'geographic_division', 'census_region',
       'county', 'experience', 'supplies', 'insured', 'involved',
       'learned_routes', 'made_plan', 'made_safer', 'planned_neighbors',
       'practiced_drills', 'documents', 'rainy_day', 'alerts',
       'family_communication', 'none', 'dont_know', 'age', 'sex', 'education',
       'race', 'homeownership', 'income', 'rentmortgage', 'rurality',
       'hazard_weight', 'geometry', 'zip_count'],
      dtype='object')

In [3]:
path = '/Users/philip/Documents/ESE/ESE_thesis/flood_experience/data/export/clean_k.csv'
df_k = pd.read_csv(path)
df_k.columns

Index(['id', 'state', 'zipcode', 'geographic_division', 'census_region',
       'county', 'awareness', 'perception', 'experience', 'floodzone',
       'efficacy', 'supplies', 'insured', 'involved', 'learned_routes',
       'made_plan', 'made_safer', 'planned_neighbors', 'practiced_drills',
       'documents', 'rainy_day', 'alerts', 'family_communication', 'none',
       'dont_know', 'age', 'sex', 'education', 'race', 'homeownership',
       'income', 'rentmortgage', 'rurality', 'hazard_weight', 'geometry',
       'zip_count'],
      dtype='object')

#### We will go through these outcome variables one by one

structural adaptation: 
- made_safer

non-structural adaptation: 
- insurance
- learned_rountes
- supplies
- involved
- made_plan
- practiced_drills
- alerts
- family_communication

In [4]:
def probit (functions, determinant, data):
    # create a dataframe to capture results 
    results = pd.DataFrame(columns=['effect', 'p', 'marginal_effect', 'pseudoR_2', 'LLRp','BIC']) 
    # iterate over functions and compute models and capture results   
    for idx, var in enumerate(functions):
        model = smf.probit(formula=var, data=data).fit(disp=0) # run the model
        #fill the dataframe columns 
        results.at[var,'effect'] = model.params[determinant]
        results.at[var,'p'] = model.pvalues[determinant]
        temporary = model.get_margeff().summary_frame() 
        results.at[var,'marginal_effect'] = temporary.at[determinant,'dy/dx']
        results.at[var,'pseudoR_2'] = model.prsquared
        results.at[var,'LLRp'] = model.llr_pvalue
        results.at[var,'BIC'] = model.bic
    return results

In [5]:
# duplicate function but with logit regression 
def logit(functions, determinant, data):
    results = pd.DataFrame(columns=['effect', 'p', 'marginal_effect', 'odds','pseudoR_2', 'LLRp', 'BIC']) #capture also odds ratio
    for idx, var in enumerate(functions):
        model = smf.logit(formula=var, data=data).fit(disp=0) #change here
        results.at[var,'effect'] = model.params[determinant]
        results.at[var,'p'] = model.pvalues[determinant]
        temporary = model.get_margeff().summary_frame() 
        results.at[var,'marginal_effect'] = temporary.at[determinant,'dy/dx']
        results.at[var, 'odds'] = np.exp(model.params[determinant]) # odds
        results.at[var,'pseudoR_2'] = model.prsquared
        results.at[var,'LLRp'] = model.llr_pvalue
        results.at[var,'BIC'] = model.bic
    return results

## What is the effect of experience on preparedness?

In [6]:
determinant = 'experience'
experience = [
    'made_safer ~ experience',
    'insured ~ experience',
    'learned_routes ~ experience',
    'supplies ~ experience',
    'involved ~ experience',
    'made_plan ~ experience', 
    'practiced_drills ~ experience', 
    'alerts ~ experience', 
    'family_communication ~ experience'
]

In [7]:
probit(functions=experience, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ experience,0.464811,0.001247,0.120893,0.022813,0.001337,453.145341
insured ~ experience,0.601947,0.000016,0.168515,0.03767,0.000017,483.963375
learned_routes ~ experience,0.789854,0.0,0.221785,0.064543,0.0,484.979794
supplies ~ experience,0.925101,0.0,0.248582,0.089055,0.0,465.787761
involved ~ experience,0.84915,0.000001,0.12531,0.087316,0.000001,270.939926
made_plan ~ experience,0.611177,0.000011,0.172844,0.038711,0.000012,488.407564
practiced_drills ~ experience,0.612721,0.000456,0.087557,0.045882,0.000497,264.515054
alerts ~ experience,0.475334,0.00091,0.125247,0.023791,0.000976,458.457243
family_communication ~ experience,0.488655,0.003015,0.084481,0.027832,0.003292,314.05451


In [8]:
logit(functions=experience, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ experience,0.810077,0.001106,0.118961,2.248082,0.022813,0.001337,453.145341
insured ~ experience,1.025431,0.000014,0.165249,2.788296,0.03767,0.000017,483.963375
learned_routes ~ experience,1.333701,0.0,0.216381,3.795062,0.064543,0.0,484.979794
supplies ~ experience,1.567844,0.0,0.241828,4.796296,0.089055,0.0,465.787761
involved ~ experience,1.653549,0.000001,0.126029,5.22549,0.087316,0.000001,270.939926
made_plan ~ experience,1.038353,0.000009,0.169445,2.824561,0.038711,0.000012,488.407564
practiced_drills ~ experience,1.210127,0.000424,0.086999,3.353909,0.045882,0.000497,264.515054
alerts ~ experience,0.825568,0.000803,0.123213,2.283178,0.023791,0.000976,458.457243
family_communication ~ experience,0.931558,0.00271,0.083393,2.538462,0.027832,0.003292,314.05451


## What is the effect of awareness on preparedness?

In [11]:
determinant = 'awareness'
awareness = [
    'made_safer ~ awareness',
    'insured ~ awareness',
    'learned_routes ~ awareness',
    'supplies ~ awareness',
    'involved ~ awareness',
    'made_plan ~ awareness', 
    'practiced_drills ~ awareness', 
    'alerts ~ awareness', 
    'family_communication ~ awareness'
]

In [12]:
probit(functions=awareness, determinant=determinant, data=df_k)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ awareness,0.639474,0.00003,0.166295,0.047884,0.000022,370.706283
insured ~ awareness,0.751019,0.0,0.217462,0.064209,0.0,408.127214
learned_routes ~ awareness,0.899099,0.0,0.260249,0.0896,0.0,407.601169
supplies ~ awareness,0.977728,0.0,0.269116,0.104284,0.0,389.071228
involved ~ awareness,0.780937,0.00006,0.129191,0.070491,0.000025,246.611062
made_plan ~ awareness,0.814094,0.0,0.231645,0.074677,0.0,401.537692
practiced_drills ~ awareness,0.728353,0.000344,0.107448,0.062673,0.000172,223.021964
alerts ~ awareness,0.817084,0.0,0.211267,0.075474,0.0,368.135285
family_communication ~ awareness,0.767772,0.000047,0.134218,0.068361,0.000021,259.116877


In [14]:
logit(functions=awareness, determinant=determinant, data=df_k)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ awareness,1.138511,0.000045,0.168648,3.122117,0.047884,0.000022,370.706283
insured ~ awareness,1.297358,0.000001,0.219653,3.659615,0.064209,0.0,408.127214
learned_routes ~ awareness,1.549804,0.0,0.263487,4.710546,0.0896,0.0,407.601169
supplies ~ awareness,1.709401,0.0,0.274782,5.525652,0.104284,0.0,389.071228
involved ~ awareness,1.563394,0.000143,0.136083,4.775,0.070491,0.000025,246.611062
made_plan ~ awareness,1.412301,0.0,0.234666,4.105392,0.074677,0.0,401.537692
practiced_drills ~ awareness,1.494508,0.000664,0.113037,4.457143,0.062673,0.000172,223.021964
alerts ~ awareness,1.456931,0.0,0.216079,4.292763,0.075474,0.0,368.135285
family_communication ~ awareness,1.515869,0.000108,0.140713,4.553377,0.068361,0.000021,259.116877


## What is the effect of insurance on preparedness?

In [17]:
determinant = 'insured'
insurance = [
    'made_safer ~ insured',
    'learned_routes ~ insured',
    'supplies ~ insured',
    'involved ~ insured',
    'made_plan ~ insured', 
    'practiced_drills ~ insured', 
    'alerts ~ insured', 
    'family_communication ~ insured'
]

In [19]:
probit(functions=insurance, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ insured,0.956,0.0,0.231244,0.088262,0.0,423.61985
learned_routes ~ insured,1.303858,0.0,0.328561,0.154144,0.0,439.706049
supplies ~ insured,0.897724,0.0,0.245253,0.074137,0.0,473.213987
involved ~ insured,0.945434,0.0,0.137373,0.100804,0.0,267.117783
made_plan ~ insured,0.727625,0.000001,0.203525,0.048446,0.000001,483.586403
practiced_drills ~ insured,0.999336,0.0,0.132852,0.114721,0.0,246.318736
alerts ~ insured,1.122672,0.0,0.264855,0.121296,0.0,413.895833
family_communication ~ insured,1.071413,0.0,0.166641,0.126718,0.0,283.362125


In [20]:
logit(functions=insurance, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ insured,1.640367,0.0,0.222688,5.157062,0.088262,0.0,423.61985
learned_routes ~ insured,2.181974,0.0,0.310753,8.863787,0.154144,0.0,439.706049
supplies ~ insured,1.507068,0.0,0.236221,4.51348,0.074137,0.0,473.213987
involved ~ insured,1.810345,0.0,0.135867,6.112554,0.100804,0.0,267.117783
made_plan ~ insured,1.225434,0.000001,0.197303,3.405645,0.048446,0.000001,483.586403
practiced_drills ~ insured,1.945666,0.0,0.132552,6.998291,0.114721,0.0,246.318736
alerts ~ insured,1.919552,0.0,0.253665,6.817901,0.121296,0.0,413.895833
family_communication ~ insured,2.009822,0.0,0.164789,7.461988,0.126718,0.0,283.362125


## What is the effect of efficacy on preparedness?

In [26]:
determinant = 'efficacy'
efficacy = [
    'made_safer ~ efficacy',
    'insured ~ efficacy',
    'learned_routes ~ efficacy',
    'supplies ~ efficacy',
    'involved ~ efficacy',
    'made_plan ~ efficacy', 
    'practiced_drills ~ efficacy', 
    'alerts ~ efficacy', 
    'family_communication ~ efficacy'
]

In [27]:
probit(functions=efficacy, determinant=determinant, data=df_k)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ efficacy,0.618903,0.000039,0.161298,0.045396,0.000035,371.643964
insured ~ efficacy,1.024123,0.0,0.278832,0.117316,0.0,385.641317
learned_routes ~ efficacy,0.923672,0.0,0.265106,0.095984,0.0,404.826342
supplies ~ efficacy,1.006819,0.0,0.273673,0.11373,0.0,385.093695
involved ~ efficacy,0.876366,0.000003,0.141493,0.093202,0.000001,240.87647
made_plan ~ efficacy,1.006819,0.0,0.273673,0.11373,0.0,385.093695
practiced_drills ~ efficacy,0.876258,0.00001,0.125175,0.09393,0.000004,215.982035
alerts ~ efficacy,0.716281,0.000002,0.188156,0.060185,0.000001,374.026605
family_communication ~ efficacy,0.750221,0.000025,0.130845,0.069646,0.000017,258.775914


In [28]:
logit(functions=efficacy, determinant=determinant, data=df_k)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ efficacy,1.085801,0.000044,0.160892,2.961812,0.045396,0.000035,371.643964
insured ~ efficacy,1.754945,0.0,0.277336,5.783133,0.117316,0.0,385.641317
learned_routes ~ efficacy,1.566545,0.0,0.262693,4.790069,0.095984,0.0,404.826342
supplies ~ efficacy,1.727221,0.0,0.272344,5.625,0.11373,0.0,385.093695
involved ~ efficacy,1.723372,0.000007,0.147085,5.60339,0.093202,0.000001,240.87647
made_plan ~ efficacy,1.727221,0.0,0.272344,5.625,0.11373,0.0,385.093695
practiced_drills ~ efficacy,1.773493,0.000026,0.13119,5.891393,0.09393,0.000004,215.982035
alerts ~ efficacy,1.25112,0.000002,0.187807,3.494253,0.060185,0.000001,374.026605
family_communication ~ efficacy,1.447762,0.00004,0.133754,4.253585,0.069646,0.000017,258.775914


# Socioeconoimc predictors 

## What is the effect of age on preparedness?

In [29]:
determinant = 'age'
age = [
    'made_safer ~ age',
    'insured ~ age',
    'learned_routes ~ age',
    'supplies ~ age',
    'involved ~ age',
    'made_plan ~ age', 
    'practiced_drills ~ age', 
    'alerts ~ age', 
    'family_communication ~ age'
]

In [30]:
probit(functions=age, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ age,-0.00319,0.341093,-0.000848,0.002016,0.340209,462.527096
insured ~ age,-0.005952,0.070502,-0.001722,0.006737,0.069199,499.123928
learned_routes ~ age,-0.000279,0.930543,-0.000084,0.000015,0.930537,517.584083
supplies ~ age,-0.009953,0.002888,-0.002894,0.018247,0.00258,501.036472
involved ~ age,-0.019071,0.000087,-0.002888,0.061931,0.000028,278.133354
made_plan ~ age,-0.006485,0.049536,-0.001895,0.007881,0.048194,503.676812
practiced_drills ~ age,-0.0178,0.000302,-0.002519,0.055657,0.000125,261.931101
alerts ~ age,0.002488,0.454015,0.000671,0.001226,0.454188,468.769866
family_communication ~ age,-0.009821,0.01729,-0.001713,0.018916,0.015391,316.821647


In [31]:
logit(functions=age, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ age,-0.005728,0.339569,-0.00086,0.994288,0.002039,0.337458,462.516677
insured ~ age,-0.010313,0.072456,-0.001723,0.98974,0.006712,0.069715,499.136125
learned_routes ~ age,-0.000478,0.930739,-0.000084,0.999523,0.000015,0.930724,517.584124
supplies ~ age,-0.017122,0.00332,-0.002886,0.983024,0.018076,0.002702,501.12151
involved ~ age,-0.0369,0.000149,-0.002884,0.963773,0.059784,0.000039,278.741823
made_plan ~ age,-0.011097,0.05214,-0.001878,0.988964,0.007781,0.049642,503.726512
practiced_drills ~ age,-0.035315,0.000456,-0.002535,0.965301,0.054204,0.000154,262.315329
alerts ~ age,0.004383,0.45392,0.00067,1.004393,0.001225,0.454399,468.77039
family_communication ~ age,-0.01913,0.018946,-0.001726,0.981052,0.018798,0.015714,316.858217


## What is the effect of income on preparedness? 

In [32]:
determinant = 'income'
income = [
    'made_safer ~ income',
    'insured ~ income',
    'learned_routes ~ income',
    'supplies ~ income',
    'involved ~ income',
    'made_plan ~ income', 
    'practiced_drills ~ income', 
    'alerts ~ income', 
    'family_communication ~ income'
]

In [33]:
probit(functions=income, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ income,-0.000002,0.112351,-0.000001,0.005846,0.104376,460.799372
insured ~ income,0.000001,0.561178,0.0,0.000685,0.562339,502.090175
learned_routes ~ income,0.0,0.978145,0.0,0.000001,0.978148,517.590931
supplies ~ income,-0.000002,0.222024,-0.0,0.003078,0.215794,508.587644
involved ~ income,0.0,0.911593,0.0,0.000043,0.911763,295.670313
made_plan ~ income,-0.0,0.960379,-0.0,0.000005,0.960362,507.577555
practiced_drills ~ income,-0.000001,0.600766,-0.0,0.001061,0.596322,276.362338
alerts ~ income,-0.0,0.86155,-0.0,0.000067,0.86129,469.299514
family_communication ~ income,0.0,0.926967,0.0,0.000027,0.927074,322.684476


In [34]:
logit(functions=income, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ income,-0.000004,0.120829,-0.000001,0.999996,0.005762,0.106906,460.837367
insured ~ income,0.000001,0.557793,0.0,1.000001,0.000689,0.561065,502.087982
learned_routes ~ income,0.0,0.977877,0.0,1.0,0.000002,0.977887,517.590913
supplies ~ income,-0.000003,0.229878,-0.0,0.999997,0.00304,0.218613,508.606316
involved ~ income,0.0,0.912324,0.0,1.0,0.000042,0.912646,295.670559
made_plan ~ income,-0.0,0.96089,-0.0,1.0,0.000005,0.960859,507.577617
practiced_drills ~ income,-0.000002,0.608486,-0.0,0.999998,0.001033,0.601249,276.3698
alerts ~ income,-0.0,0.864562,-0.0,1.0,0.000064,0.864115,469.300757
family_communication ~ income,0.0,0.92797,0.0,1.0,0.000026,0.928169,322.684727


## What is the effect of education on preparedness? 

In [35]:
determinant = 'education'
education = [
    'made_safer ~ education',
    'insured ~ education',
    'learned_routes ~ education',
    'supplies ~ education',
    'involved ~ education',
    'made_plan ~ education', 
    'practiced_drills ~ education', 
    'alerts ~ education', 
    'family_communication ~ education'
]

In [36]:
probit(functions=education, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ education,-0.05668,0.2219,-0.015045,0.003318,0.221172,461.939987
insured ~ education,0.060118,0.181305,0.017451,0.003657,0.18063,500.633411
learned_routes ~ education,-0.008668,0.845351,-0.00261,0.000075,0.845344,517.553632
supplies ~ education,-0.063396,0.154212,-0.01871,0.004093,0.153455,508.082209
involved ~ education,0.039417,0.476756,0.006337,0.00179,0.476346,295.175376
made_plan ~ education,-0.045045,0.314604,-0.013247,0.002046,0.31416,506.566921
practiced_drills ~ education,0.006561,0.908007,0.00098,0.000051,0.90801,276.629565
alerts ~ education,0.018098,0.6951,0.004887,0.000336,0.695077,469.176401
family_communication ~ education,0.05513,0.309258,0.009764,0.003344,0.308331,321.655043


In [37]:
logit(functions=education, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ education,-0.10111,0.221448,-0.015153,0.903833,0.003337,0.219833,461.931277
insured ~ education,0.103883,0.182061,0.017406,1.10947,0.003648,0.181151,500.637689
learned_routes ~ education,-0.01476,0.846055,-0.002587,0.985349,0.000075,0.846032,517.553974
supplies ~ education,-0.110679,0.152878,-0.018931,0.895226,0.004137,0.151265,508.060324
involved ~ education,0.080964,0.469341,0.006556,1.084331,0.001851,0.468863,295.157938
made_plan ~ education,-0.0776,0.316039,-0.01321,0.925335,0.002038,0.315022,506.570522
practiced_drills ~ education,0.013786,0.906127,0.00102,1.013881,0.000053,0.906135,276.629013
alerts ~ education,0.031807,0.695711,0.004865,1.032319,0.000335,0.695703,469.177064
family_communication ~ education,0.107566,0.308527,0.009805,1.113564,0.003356,0.30744,321.651212


## What is the effect of homeownership on preparedness?

In [38]:
determinant = 'homeownership'
homeownership = [
    'made_safer ~ homeownership',
    'insured ~ homeownership',
    'learned_routes ~ homeownership',
    'supplies ~ homeownership',
    'involved ~ homeownership',
    'made_plan ~ homeownership', 
    'practiced_drills ~ homeownership', 
    'alerts ~ homeownership', 
    'family_communication ~ homeownership'
]

In [39]:
probit(functions=homeownership, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ homeownership,0.15319,0.260033,0.040681,0.002825,0.258898,462.162099
insured ~ homeownership,0.330079,0.013286,0.094931,0.012682,0.012662,496.21012
learned_routes ~ homeownership,-0.068027,0.596659,-0.020477,0.000554,0.596768,517.311783
supplies ~ homeownership,0.03686,0.776275,0.010922,0.000162,0.776208,510.038965
involved ~ homeownership,-0.389712,0.018464,-0.061582,0.019883,0.017613,290.048307
made_plan ~ homeownership,-0.063126,0.626187,-0.018595,0.000479,0.62629,507.3429
practiced_drills ~ homeownership,-0.06253,0.710084,-0.009336,0.000522,0.710245,276.504884
alerts ~ homeownership,0.082894,0.538151,0.02237,0.000831,0.537747,468.950314
family_communication ~ homeownership,-0.031037,0.843614,-0.005514,0.000125,0.843666,322.653963


In [40]:
logit(functions=homeownership, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ homeownership,0.272081,0.261626,0.040797,1.312693,0.002825,0.258898,462.162099
insured ~ homeownership,0.575817,0.014206,0.095597,1.778582,0.012682,0.012662,496.21012
learned_routes ~ homeownership,-0.116808,0.596486,-0.020465,0.889756,0.000554,0.596768,517.311783
supplies ~ homeownership,0.063614,0.776377,0.010926,1.065681,0.000162,0.776208,510.038965
involved ~ homeownership,-0.775904,0.019567,-0.062144,0.460288,0.019883,0.017613,290.048307
made_plan ~ homeownership,-0.10899,0.626026,-0.018584,0.896739,0.000479,0.62629,507.3429
practiced_drills ~ homeownership,-0.126082,0.70992,-0.009331,0.881543,0.000522,0.710245,276.504884
alerts ~ homeownership,0.14651,0.538725,0.022399,1.157786,0.000831,0.537747,468.950314
family_communication ~ homeownership,-0.060336,0.843555,-0.005511,0.941448,0.000125,0.843666,322.653963


## What is the effect of monthly rent or mortgage payments on preparedness? 

In [41]:
determinant = 'rentmortgage'
rentmortgage = [
    'made_safer ~ rentmortgage',
    'insured ~ rentmortgage',
    'learned_routes ~ rentmortgage',
    'supplies ~ rentmortgage',
    'involved ~ rentmortgage',
    'made_plan ~ rentmortgage', 
    'practiced_drills ~ rentmortgage', 
    'alerts ~ rentmortgage', 
    'family_communication ~ rentmortgage'
]

In [42]:
probit(functions=rentmortgage, determinant=determinant, data=df_n)

,effect,p,marginal_effect,pseudoR_2,LLRp,BIC
made_safer ~ rentmortgage,0.000072,0.373801,0.000019,0.001751,0.374086,462.646694
insured ~ rentmortgage,0.000113,0.156845,0.000033,0.004086,0.157037,500.423318
learned_routes ~ rentmortgage,0.000082,0.292847,0.000025,0.002187,0.29313,516.486492
supplies ~ rentmortgage,0.000132,0.092087,0.000039,0.005701,0.092073,507.282027
involved ~ rentmortgage,0.000115,0.251025,0.000018,0.004639,0.251593,294.368154
made_plan ~ rentmortgage,0.000131,0.095671,0.000038,0.00561,0.095536,504.801497
practiced_drills ~ rentmortgage,0.000289,0.005026,0.000042,0.030355,0.004617,268.619103
alerts ~ rentmortgage,0.000057,0.483418,0.000015,0.001073,0.483779,468.839711
family_communication ~ rentmortgage,0.000224,0.018865,0.000039,0.017892,0.018445,317.13948


In [43]:
logit(functions=rentmortgage, determinant=determinant, data=df_n)

,effect,p,marginal_effect,odds,pseudoR_2,LLRp,BIC
made_safer ~ rentmortgage,0.000128,0.372142,0.000019,1.000128,0.001758,0.373228,462.643851
insured ~ rentmortgage,0.000191,0.16079,0.000032,1.000191,0.003999,0.161509,500.465801
learned_routes ~ rentmortgage,0.00014,0.295147,0.000024,1.00014,0.002162,0.295987,516.499494
supplies ~ rentmortgage,0.000226,0.093139,0.000039,1.000226,0.005651,0.093512,507.306918
involved ~ rentmortgage,0.00022,0.25757,0.000018,1.00022,0.004484,0.259638,294.411908
made_plan ~ rentmortgage,0.000226,0.094777,0.000038,1.000226,0.005623,0.095171,504.795371
practiced_drills ~ rentmortgage,0.000573,0.005019,0.000042,1.000573,0.030097,0.004794,268.687534
alerts ~ rentmortgage,0.000099,0.486414,0.000015,1.000099,0.001056,0.487275,468.847506
family_communication ~ rentmortgage,0.00043,0.018914,0.000039,1.00043,0.017749,0.01892,317.183967
